In [1]:
import pandas as pd
from sklearn import preprocessing


In [2]:
df = pd.read_csv(
    "./data/interim/transactions.csv",
    index_col=0,
    parse_dates=[
        "transactionDateTime",
        "currentExpDate",
        "accountOpenDate",
        "dateOfLastAddressChange",
    ],
)


# Adding Columns

In [4]:
df.columns


Index(['customerId', 'creditLimit', 'availableMoney', 'transactionDateTime',
       'transactionAmount', 'merchantName', 'acqCountry',
       'merchantCountryCode', 'posEntryMode', 'posConditionCode',
       'merchantCategoryCode', 'currentExpDate', 'accountOpenDate',
       'dateOfLastAddressChange', 'cardCVV', 'enteredCVV', 'cardLast4Digits',
       'transactionType', 'currentBalance', 'cardPresent',
       'expirationDateKeyInMatch', 'isFraud'],
      dtype='object')

In [5]:
# perhaps the Age of the account is important
df["accountAge"] = (df.transactionDateTime - df.accountOpenDate).dt.days


In [6]:
df[["transactionType", "isFraud"]].value_counts()


transactionType       isFraud
PURCHASE              False      725108
ADDRESS_VERIFICATION  False       19856
REVERSAL              False       19738
PURCHASE              True        11528
REVERSAL              True          324
ADDRESS_VERIFICATION  True          114
dtype: int64

In [7]:
df[df.cardCVV != df.enteredCVV]["isFraud"].value_counts(normalize=True)



False    0.971429
True     0.028571
Name: isFraud, dtype: float64

In [8]:
# Card CVV vs entered CVV
df["cvvMatch"] = df.cardCVV == df.enteredCVV


In [9]:
# time since address was changed
df["sinceDateOfLastAddressChange"] = (
    df.transactionDateTime - df.dateOfLastAddressChange
).dt.days


In [10]:
# If country codes match
df["countryMatch"] = df.acqCountry == df.merchantCountryCode


In [11]:
# General date properties
df["dayOfMonth"] = df.transactionDateTime.dt.day
df["month"] = df.transactionDateTime.dt.month
df["dayOfYear"] = df.transactionDateTime.dt.dayofyear
df["weekOfYear"] = df.transactionDateTime.dt.weekofyear
df["dayOfWeek"] = df.transactionDateTime.dt.dayofweek
df["quarter"] = df.transactionDateTime.dt.quarter
df["hour"] = df.transactionDateTime.dt.hour


<ipython-input-11-381f3c90c286>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["weekOfYear"] = df.transactionDateTime.dt.weekofyear


In [12]:
# weekday or weekend
df["weekday"] = df.dayOfWeek < 5


In [13]:
# time of day
df["timeOfDay"] = pd.cut(df.hour, bins=3, labels=[0, 1, 2])


## Aggregations based on client ID

In [14]:
# Creating Cumulative Mean. Shifted so each line shows what's happened in the past
df["cumMean"] = (
    df.groupby("customerId")["isFraud"]
    .apply(lambda x: x.shift().expanding().mean())
    .fillna(0)
)


## Revisting with more features
## After running a basic Log Regression model, I think I need better features

### Transactions that happen less than 3 minutes apart

In [15]:
df["quick_transaction"] = (
    df.groupby("customerId").apply(
        lambda x: x.rolling("3min", on="transactionDateTime").count()
    )
)["isFraud"]


### Transactions that happen in separate countries, but minutes apart

In [16]:
df["time_shift_down1"] = df.groupby("customerId").transactionDateTime.shift(1)
df["country_shift_down1"] = df.groupby("customerId").acqCountry.shift(1)


In [17]:
ctry_diff = df.acqCountry != df.country_shift_down1
in_window = (df.transactionDateTime - df.time_shift_down1) < pd.Timedelta("5 minutes")
df["ctry_diff_five_min"] = (ctry_diff & in_window).fillna(False)


### Minimum time (minutes) between transactions in a 30 day span

In [18]:
# Difference in time per group
df["time_diff"] = (
    df.groupby("customerId")["transactionDateTime"].diff().fillna(pd.Timedelta(0))
)
df["time_diff"] = (df.time_diff.dt.total_seconds() / 60).astype("int")


In [19]:
df["min_time_diff_month"] = (
    df.groupby("customerId")
    .apply(lambda x: x.rolling("30D", on="transactionDateTime")["time_diff"].min())
    .reset_index("customerId")
    .drop("customerId", axis=1)
    .sort_index()
    .squeeze()
)


### Has Fraud in the past

In [20]:
df["cumFraud"] = (
    df.groupby("customerId")["isFraud"]
    .apply(lambda x: x.shift().expanding().sum())
    .fillna(0)
)


### Average Amount spent per transaction over 1 month

In [21]:
df["avg_spent_month"] = (
    df.groupby("customerId")
    .apply(
        lambda x: x.rolling("30D", on="transactionDateTime")["transactionAmount"].mean()
    )
    .reset_index("customerId")
    .drop("customerId", axis=1)
    .sort_index()
    .squeeze()
)


### Total amount spent on the same day

In [22]:
df["total_spent_one_day"] = (
    df.groupby("customerId")
    .apply(
        lambda x: x.rolling("1D", on="transactionDateTime")["transactionAmount"].sum()
    )
    .reset_index("customerId")
    .drop("customerId", axis=1)
    .sort_index()
    .squeeze()
)


# Encoding

In [23]:
# Commented out because It takes a while to run
"""
import sweetviz as sv
my_report = sv.analyze(df)
my_report.show_html()
"""



'\nimport sweetviz as sv\nmy_report = sv.analyze(df)\nmy_report.show_html()\n'

In [24]:
"""
Enconding List

customerId - nothing
creditLimit - nothing
availableMoney - cut
transactionDateTime - drop
transactionAmount - qcut
merchantName - ordinal
acqCountry - ordinal
merchantCountryCode - ordinal
posEntryMode - ordinal
posConditionCode - ordinal
merchantCategoryCode - ordinal
currentExpDate - drop
accountOpenDate - drop
dateOfLastAddressChange - drop
cardCVV - drop
enteredCVV - drop
cardLast4Digits - drop
transactionType - ordinal
currentBalance - nothing
cardPresent - boolean
expirationDateKeyInMatch - boolean
isFraud - boolean
accountAge - nothing
cvvMatch - boolean
sinceDateOfLastAddressChange - nothing
countryMatch - boolean
dayOfMonth - nothing
month - nothing
dayOfYear - nothing
weekOfYear - nothing
dayOfWeek - nothing
quarter - nothing
hour - nothing
weekday - boolean
timeOfDay - nothing
cumMean - nothing
quick_transaction - nothing
time_shift_down1 - drop
country_shift_down1 - drop
ctry_diff_five_min - boolean
time_diff - nothing
min_time_diff_month - nothing
cumFraud - nothing
avg_spent_month - nothing
total_spent_one_day - nothing
"""


'\nEnconding List\n\ncustomerId - nothing\ncreditLimit - nothing\navailableMoney - cut\ntransactionDateTime - drop\ntransactionAmount - qcut\nmerchantName - ordinal\nacqCountry - ordinal\nmerchantCountryCode - ordinal\nposEntryMode - ordinal\nposConditionCode - ordinal\nmerchantCategoryCode - ordinal\ncurrentExpDate - drop\naccountOpenDate - drop\ndateOfLastAddressChange - drop\ncardCVV - drop\nenteredCVV - drop\ncardLast4Digits - drop\ntransactionType - ordinal\ncurrentBalance - nothing\ncardPresent - boolean\nexpirationDateKeyInMatch - boolean\nisFraud - boolean\naccountAge - nothing\ncvvMatch - boolean\nsinceDateOfLastAddressChange - nothing\ncountryMatch - boolean\ndayOfMonth - nothing\nmonth - nothing\ndayOfYear - nothing\nweekOfYear - nothing\ndayOfWeek - nothing\nquarter - nothing\nhour - nothing\nweekday - boolean\ntimeOfDay - nothing\ncumMean - nothing\nquick_transaction - nothing\ntime_shift_down1 - drop\ncountry_shift_down1 - drop\nctry_diff_five_min - boolean\ntime_diff - n

## Cut

In [25]:
# available money - cut
df["availableMoney"] = pd.cut(
    df.availableMoney,
    bins=[-5000, -1000, -500, -100, 0, 100, 500, 1000, 5000, 50000],
    labels=[0, 1, 2, 3, 4, 5, 6, 7, 8],
)

# transactionAmount - qcut
df["transactionAmount"] = pd.qcut(df.transactionAmount, 4, labels=[0, 1, 2, 3])


## Ordinal

In [26]:
# merchantName - i'm going to remove the individual locations - like AMC #010101 - could change this choice later
df["merchantName"] = df.merchantName.str.replace(r"\s#.*$", "", regex=True)

# List of cols to encode
ordinal_encode = [
    "acqCountry",
    "merchantCountryCode",
    "posEntryMode",
    "posConditionCode",
    "merchantCategoryCode",
    "transactionType",
    "merchantName",
]

# Encode all cols
ordinal = preprocessing.OrdinalEncoder()
df[ordinal_encode] = ordinal.fit_transform(df[ordinal_encode])
# Counts numpy of nans
df[ordinal_encode].isnull().sum().sum()


0

## Booleans

In [27]:
boolean_encode = [
    "cardPresent",
    "expirationDateKeyInMatch",
    "isFraud",
    "cvvMatch",
    "countryMatch",
    "weekday",
    "ctry_diff_five_min",
]

df[boolean_encode] = df[boolean_encode].astype("int")


## Drop

In [28]:
df = df.drop(
    [
        "transactionDateTime",
        "currentExpDate",
        "accountOpenDate",
        "dateOfLastAddressChange",
        "cardCVV",
        "enteredCVV",
        "cardLast4Digits",
        "time_shift_down1",
        "country_shift_down1",
    ],
    axis=1,
)


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 776668 entries, 0 to 786362
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype   
---  ------                        --------------   -----   
 0   customerId                    776668 non-null  int64   
 1   creditLimit                   776668 non-null  float64 
 2   availableMoney                776668 non-null  category
 3   transactionAmount             776668 non-null  category
 4   merchantName                  776668 non-null  float64 
 5   acqCountry                    776668 non-null  float64 
 6   merchantCountryCode           776668 non-null  float64 
 7   posEntryMode                  776668 non-null  float64 
 8   posConditionCode              776668 non-null  float64 
 9   merchantCategoryCode          776668 non-null  float64 
 10  transactionType               776668 non-null  float64 
 11  currentBalance                776668 non-null  float64 
 12  cardPresent                   

In [33]:
df.availableMoney

0         7
1         7
2         7
3         7
4         7
         ..
786358    8
786359    8
786360    8
786361    8
786362    8
Name: availableMoney, Length: 776668, dtype: category
Categories (9, int64): [0 < 1 < 2 < 3 ... 5 < 6 < 7 < 8]

In [29]:
# Saves types to load quickly
df.dtypes.to_csv("./data/processed/transactions_dtypes.csv")


In [30]:
df.to_csv("./data/processed/transactions.csv")
